# 14. Train Final Model: LogReg

In [1]:
import pandas as pd
import os

# Import Data

In [2]:
# Import Data
Train_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XTrain_16-1_16-5.csv')
Train_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yTrain_16-1_16-5.csv')
XTrain = Train_data_X
yTrain = Train_data_y['Max_PULocationID']

In [3]:
Val_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XVal_16-5_16-6.csv')
Val_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yVal_16-5_16-6.csv')
XVal = Val_data_X
yVal = Val_data_y['Max_PULocationID']

In [4]:
Test_data_X = pd.read_csv('../data/curated/ModelBuilding/Discrete/XTest_16-6_16-6.csv')
Test_data_y = pd.read_csv('../data/curated/ModelBuilding/Discrete/yTest_16-6_16-6.csv')
XTest = Test_data_X
yTest = Test_data_y['Max_PULocationID']

# Extra One Hot Encoding (On the Fly)

In [5]:
from sklearn.preprocessing import OneHotEncoder

# OHE for DOLocationID

ohe = OneHotEncoder(handle_unknown='ignore')

Train_data_to_ohe = XTrain[['DOLocationID']]
Train_data_ohe = ohe.fit_transform(Train_data_to_ohe).toarray()

Train_data_ohe = pd.DataFrame(Train_data_ohe,
                              columns=list(ohe.get_feature_names_out(['DOLocationID'])))

XTrain = Train_data_X.drop(['DOLocationID'], axis=1)

for col in Train_data_ohe.columns:
    new_col = Train_data_ohe[col]
    new_col.index = range(len(new_col))

    XTrain[str(col)] = new_col

XTrain = XTrain.drop('datetime', axis=1)



Val_data_to_ohe = XVal[['DOLocationID']]
Val_data_ohe = ohe.transform(Val_data_to_ohe).toarray()

Val_data_ohe = pd.DataFrame(Val_data_ohe,
                            columns=list(ohe.get_feature_names_out(['DOLocationID'])))

XVal = XVal.drop(['DOLocationID'], axis=1)

for col in Val_data_ohe.columns:
    new_col = Val_data_ohe[col]
    new_col.index = range(len(new_col))

    XVal[str(col)] = new_col

XVal = XVal.drop('datetime', axis=1)



Test_data_to_ohe = XTest[['DOLocationID']]
Test_data_ohe = ohe.transform(Test_data_to_ohe).toarray()

Test_data_ohe = pd.DataFrame(Test_data_ohe,
                             columns=list(ohe.get_feature_names_out(['DOLocationID'])))

XTest = XTest.drop(['DOLocationID'], axis=1)

for col in Test_data_ohe.columns:
    new_col = Test_data_ohe[col]
    new_col.index = range(len(new_col))

    XTest[str(col)] = new_col

XTest = XTest.drop('datetime', axis=1)

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_8485/2543678202.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  XTrain[str(col)] = new_col
/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_8485/2543678202.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  XVal[str(col)] = new_col
/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_8485/2543678202.py:55: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

# Special format for getting data in right format for accuracy evaluation

In [6]:
# read lists stored as a string in csv and re-establish as lists
train_obs = [[int(x) for x in Max_IDs.strip('[]').split(',')] for Max_IDs in Train_data_y['Max_PULocationIDs']]
val_obs = [[int(x) for x in Max_IDs.strip('[]').split(',')] for Max_IDs in Val_data_y['Max_PULocationIDs']]
test_obs = [[int(x) for x in Max_IDs.strip('[]').split(',')] for Max_IDs in Test_data_y['Max_PULocationIDs']]

In [7]:
def accuracy_score(pred, obs):
    """ Helper function to get accuracy score for multiple correct labels """
    n = len(pred)
    n_tp = 0

    for i in range(n):
        if pred[i] in obs[i]:
            n_tp += 1

    return n_tp/n

# Train model and get statistics for each DOLocation

In [8]:
from sklearn.linear_model import LogisticRegression

In [17]:
# Initialise
logR = LogisticRegression(C = 10,
                          solver = 'newton-cg',
                          penalty = 'l2',
                          max_iter = 100000)

# Fit
logR.fit(XTrain, yTrain)

LogisticRegression(C=10, max_iter=100000, solver='newton-cg')

Export object (and reopen, if don't want to have to rebuild the model every time)

In [18]:
# create directory to store tuning results
output_relative_dirs = ['../objects/models']

# check if it exists as it makedir will
# raise an error if it does exist
for output_relative_dir in output_relative_dirs:
    if not os.path.exists(output_relative_dir):
        os.makedirs(output_relative_dir)

In [19]:
import pickle
with open('../objects/models/logreg.pickle', 'wb') as f:
    pickle.dump(logR,f)

In [20]:
with open('../objects/models/logreg.pickle', 'rb') as g:
    object = pickle.load(g)

# Get statistics for each DOLocationID

In [21]:
# create list to store data to log
id = list()
TrainAccu = list()
ValAccu = list()
TestAccu = list()
TrainCount = list()
ValCount = list()
TestCount = list()


for i in range(1, 264): # iterate through the DOLocations

    print(i)
    id.append(i)

    # some DOLocation did not exist for training data
    # so were never one hot encoded
    if f'DOLocationID_{i}.0' in XTrain.columns:

        # get the data for just in this Location

        XTrain_subset = XTrain[XTrain[f'DOLocationID_{i}.0']==1]
        yTrain_subset = Train_data_y[XTrain[f'DOLocationID_{i}.0']==1]

        XVal_subset = XVal[XVal[f'DOLocationID_{i}.0']==1]
        yVal_subset = Val_data_y[XVal[f'DOLocationID_{i}.0']==1]

        XTest_subset = XTest[XTest[f'DOLocationID_{i}.0']==1]
        yTest_subset = Test_data_y[XTest[f'DOLocationID_{i}.0']==1]

        # preprocess data required for accuracy test with
        # multiple correct labels (just for this DOL subsample)
        train_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                            for Max_IDs in yTrain_subset['Max_PULocationIDs']]
        val_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                          for Max_IDs in yVal_subset['Max_PULocationIDs']]
        test_obs_subset = [[int(x) for x in Max_IDs.strip('[]').split(',')]
                           for Max_IDs in yTest_subset['Max_PULocationIDs']]

        TrainCount.append(len(XTrain_subset))
        ValCount.append(len(XVal_subset))
        TestCount.append(len(XTest_subset))


        # use tests to avoid errors when there are 0 samples
        if len(XTrain_subset) > 0:
            # make predictions
            predTrain_subset = logR.predict(XTrain_subset)
            # get accuracy score and put it into list
            TrainAccu.append(accuracy_score(predTrain_subset, train_obs_subset))
        else:
            # else append a placeholder to allow DataFrame construction later
            TrainAccu.append(-1)

        # use tests to avoid errors when there are 0 samples
        if len(XVal_subset) > 0:
            # make predictions
            predVal_subset = logR.predict(XVal_subset)
            # get accuracy score and put it into list
            ValAccu.append(accuracy_score(predVal_subset, val_obs_subset))
        else:
            # else append a placeholder to allow DataFrame construction later
            ValAccu.append(-1)

        # use tests to avoid errors when there are 0 samples
        if len(XTest_subset) > 0:
            # make predictions
            predTest_subset = logR.predict(XTest_subset)
            # get accuracy score and put it into list
            TestAccu.append(accuracy_score(predTest_subset, test_obs_subset))
        else:
            # else append a placeholder to allow DataFrame construction later
            TestAccu.append(-1)

    else:
        # else append a placeholder to allow DataFrame construction later
        TrainAccu.append(-2)
        ValAccu.append(-2)
        TestAccu.append(-2)
        TrainCount.append(0)
        ValCount.append(0)
        TestCount.append(0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263


In [22]:
# compile the statistics into a DataFrame
accuracy_by_location = pd.DataFrame({'DOLocationID': id,
                                     'TrainAccu': TrainAccu,
                                     'ValAccu': ValAccu,
                                     'TestAccu': TestAccu,
                                     'TrainCount': TrainCount,
                                     'ValCount': ValCount,
                                     'TestCount': TestCount})
accuracy_by_location

,DOLocationID,TrainAccu,ValAccu,TestAccu,TrainCount,ValCount,TestCount
0,1,0.534913,0.537143,0.540698,802,175,172
1,2,1.000000,-1.000000,-1.000000,4,0,0
2,3,1.000000,-1.000000,-1.000000,2,0,0
3,4,0.937030,0.911628,0.888350,1064,215,206
4,5,1.000000,-1.000000,-1.000000,1,0,0
...,...,...,...,...,...,...,...
258,259,0.657895,0.200000,0.400000,38,5,5
259,260,0.549518,0.491304,0.452915,1141,230,223
260,261,0.512915,0.434211,0.412935,1084,228,201
261,262,0.744015,0.761682,0.748768,1086,214,203


# Export statistics

In [23]:
# create directory to store models results
output_relative_dirs = ['../data/curated/final_model_data']

# check if it exists as it makedir will raise an error if it does exist
for output_relative_dir in output_relative_dirs:
    if not os.path.exists(output_relative_dir):
        os.makedirs(output_relative_dir)

In [24]:
accuracy_by_location.to_csv('../data/curated/final_model_data/logreg.csv')